In [1]:
##http://data8.org/datascience/tables.html
from datascience import *
import pandas as pd ##Importing panda Library
import numpy as np
import time
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  #data_split
from sklearn.svm import SVC  #SVM
from sklearn.metrics import classification_report, confusion_matrix  #model eval
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
from sklearn.decomposition import PCA
import random

##Random Forest
from sklearn.model_selection import train_test_split #training and testing split
from sklearn.preprocessing import StandardScaler  #feature scaling
from sklearn.ensemble import RandomForestRegressor  #random forest
from sklearn import metrics  #model evaluation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier




##XGboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


%matplotlib inline
harmony = pd.read_csv("Harmony_data.csv") #Reading the CSV file (My Data Frame)
test = pd.read_csv("production_data_test.csv") #Reading the CSV file (My Data Frame)
train = pd.read_csv("production_data_train.csv") #Reading the CSV file (My Data Frame)
ihs = pd.read_csv("IHS_data.csv") #Reading the CSV file (My Data Frame)
api = pd.read_csv("test_APIs.csv", header = None) #Reading the CSV file (My Data Frame) 
api.columns = ["API"] 
test = pd.merge(left= api,right=test, left_on= 'API', right_on='API')

In [12]:
##DFS
##Remove Duplicates
train_df = train.drop_duplicates()
train_df = train_df.sort_values(by = ['API','Year', 'Month']) 
dfs = [x.reset_index(drop = True) for _,x in train_df.groupby('API')] ##Divide the data into list of dataframes
count = 0
for d in range(0,len(dfs)):
    maxx = dfs[d]["Liquid"].idxmax()
    dfs[d] = dfs[d].iloc[maxx:,:]
    indexx = np.arange(1,len(dfs[d]) + 1,1)
    dfs[d]['Month_Index'] = indexx.tolist()
train_df_data = pd.concat(dfs)

In [14]:
Apis_36 = train_df_data[train_df_data.Month_Index == 36]
new_df = pd.merge(left= train_df_data , right=Apis_36['API'], left_on= 'API', right_on='API')
my_df_sum = new_df.groupby('API').apply(lambda x: x.iloc[0:36])
my_df_sum = my_df_sum.rename(columns = {"API": "API2"})
my_df_sum = my_df_sum.groupby('API')['Liquid'].sum().reset_index()
my_df_sum

,API,Liquid
0,5001097720000,65412
1,5001097780100,84043
2,5001097800000,13471
3,5001097820000,12992
4,5001097830000,15195
...,...,...
7826,49021222940000,227454
7827,49021222950000,211789
7828,49021223030000,230485
7829,49021226570000,26615


In [15]:
merged = pd.merge(my_df_sum, ihs, on='API')
new_df = pd.merge(merged, harmony, on='API')
train_final = new_df[['API', 'Liquid', 'BasinName', 'StateName',
                   'LatWGS84', 'LonWGS84','LATERAL_LENGTH_BLEND',
                   'PROP_PER_FOOT', 'WATER_PER_FOOT',
                       'GOR_30']]
#, 'GOR_60', 'GOR_90'  'BottomHoleLatitude', 'BottomHoleLongitude',

train_final = train_final.fillna(train_final[['LatWGS84', 'LonWGS84','LATERAL_LENGTH_BLEND',
       'GOR_30', 'PROP_PER_FOOT', 'WATER_PER_FOOT']].median())

train_df_final = pd.get_dummies(train_final, columns=["BasinName", "StateName"], 
                    prefix=["basin", "state"], drop_first = True)

In [ ]:
from scipy.stats import chi2_contingency
csq=chi2_contingency(pd.crosstab(train_final['BasinName'], train_final['Liquid']))
print("P-value: ",csq[1])

# Trial

In [16]:
#training = train_df_final[~my_df_final.API.isin(api.API)]
train_x = train_df_final[train_df_final.columns.difference(['API', 'Liquid'])]
#pca = PCA(n_components=5)
#principalComponents = pca.fit_transform(train_x)
#principalDf = pd.DataFrame(data = principalComponents
            # , columns = ['principal component 1', 'principal component 2', 
                        #  'principal component 3','principal component 4', 'principal component 5'])
train_y = train_df_final.loc[:,train_df_final.columns == 'Liquid']
random.seed(9001)
x_train, x_test, y_train,y_test=train_test_split(train_x,train_y,test_size=0.2)

In [17]:
##Random Forest
x_train2 = preprocessing.normalize(x_train)
x_test2 = preprocessing.normalize(x_test)
regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
rms

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


46974.55840139909

# Training Data

In [18]:
train_x = train_df_final[train_df_final.columns.difference(['API', 'Liquid'])]
train_y = train_df_final.loc[:, train_df_final.columns == 'Liquid']
train_x = preprocessing.normalize(train_x)
regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(train_x, train_y)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

# Test Data

In [ ]:
new_df

In [19]:
merged = pd.merge(api, ihs, on='API')
new_df = pd.merge(merged, harmony, on='API')
test_final = new_df[['API', 'BasinName',
                   'LatWGS84', 'LonWGS84','LATERAL_LENGTH_BLEND',"StateName",
                   'PROP_PER_FOOT', 'WATER_PER_FOOT',
                       'GOR_30']]
#, 'GOR_60', 'GOR_90'  'BottomHoleLatitude', 'BottomHoleLongitude',

test_final = test_final.fillna(test_final[['LatWGS84', 'LonWGS84','LATERAL_LENGTH_BLEND',
       'GOR_30', 'PROP_PER_FOOT', 'WATER_PER_FOOT']].median())

test_df_final = pd.get_dummies(test_final, columns=["BasinName", "StateName"], 
                    prefix=["basin", "state"], drop_first = True)

test_x = test_df_final[test_df_final.columns.difference(['API'])]
#test_y = test_df_final.loc[:, test_df_final.columns == 'Liquid']
test_x = preprocessing.normalize(test_x)
y_pred = regressor.predict(test_x)
y_pred

array([202620.615, 169515.89 , 169500.705, ..., 101542.91 , 170008.455,
        55756.46 ])

In [20]:
##Fun 
test_df = test.drop_duplicates()
test_df = test_df.sort_values(by = ['API','Year', 'Month']) 

dfs = [x.reset_index(drop = True) for _,x in test_df.groupby('API')] ##Divide the data into list of dataframes
count = 0
for d in range(0,len(dfs)):
    maxx = dfs[d]["Liquid"].idxmax()
    dfs[d] = dfs[d].iloc[maxx:,:]
    indexx = np.arange(1,len(dfs[d]) + 1,1)
    dfs[d]['Month_Index'] = indexx.tolist()
test_df_data = pd.concat(dfs)

Apis_36 = test_df_data[test_df_data.Month_Index == 36]
new_df = pd.merge(left= test_df_data , right=Apis_36['API'], left_on= 'API', right_on='API')
test_df_data = new_df.groupby('API').apply(lambda x: x.iloc[0:36])
my_df_sum = test_df_data.rename(columns = {"API": "API2"})
my_df_sum = my_df_sum.groupby('API')['Liquid'].sum().reset_index()
ready = pd.merge(left= my_df_sum , right=api['API'], left_on= 'API', right_on='API')



In [26]:
##Compare the APIS that reached 36 in test with our predictions
api["Predicted"] = y_pred
compares = pd.merge(api, ready, how="left", on='API').dropna() 
rms = sqrt(mean_squared_error(compares.Predicted, compares.Liquid))
rms

72729.61774531697

In [ ]:
##Save
api["Predicted"] = y_pred
api.columns = ["Id","Predicted"] 
api.to_csv('PREDICTIONS.csv', index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
CV_rfc.fit(train_x, train_y)
CV_rfc.best_params_